##01. Imports and Configuration

What: Set up imports, constants, seeds, paths, and lightweight utility/provenance writers.
Why: Ensures fixed environment, deterministic behaviour, and manifests without altering any modelling code.
Method choices: Fixed splits (Train 2021-02-03→2022-12-30; Val 2023-01-03→2023-05-31; Test 2023-06-01→2023-12-28), Target = Close.shift(-1), America/New_York with 16:00 cut-off, Train-only scaling for deep models, early stopping on Validation; deep ablations use frozen weights on Test (no monthly refit; expanding-origin is used for ARIMA/ARIMAX, not here).

In [ ]:
import os, sys, json, math, time, hashlib, platform, warnings, shutil, random, io
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple, Dict, List, Optional

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

SCHEMA_VERSION = "1.0"

TRAIN_START = "2021-02-03"
TRAIN_END   = "2022-12-30"
VAL_START   = "2023-01-03"
VAL_END     = "2023-05-31"
TEST_START  = "2023-06-01"
TEST_END    = "2023-12-28"
TEST_DAYS_EXPECTED = 146

LOOKBACK_L = 90
HORIZON = 1

BATCH_TRAIN = 64
BATCH_VAL   = 32
MAX_EPOCHS  = 300
PATIENCE    = 12
LR          = 1e-3
GRAD_CLIP   = 0.5

HIDDEN_SIZE = 64
NUM_LAYERS  = 2
DROPOUT     = 0.20
BIDIR       = False

SEED = 42
EPSILON_RET = 0.0010

INPUT_DIR = Path("final_inputs")
TICKERS = ["AAPL","AMZN","MSFT","TSLA","AMD"]

REQUIRED_COLS = ["date","ticker","Open","High","Low","Close","Volume"]

def set_all_seeds(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def env_manifest(model_id: str, extra: Optional[Dict] = None) -> Dict[str, str]:
    env = {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "numpy": np.__version__,
        "pandas": pd.__version__,
        "torch": torch.__version__,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "timezone": "America/New_York",
        "market_close": "16:00",
        "model_id": model_id,
        "seed": SEED,
    }
    try:
        import sklearn
        env["sklearn"] = sklearn.__version__
    except Exception:
        env["sklearn"] = "unknown"
    if env["device"] == "cuda":
        try:
            env["cuda_name"] = torch.cuda.get_device_name(0)
            env["cuda_capability"] = ".".join(map(str, torch.cuda.get_device_capability(0)))
        except Exception:
            pass
    if extra:
        env.update(extra)
    return env

def write_run_root_manifests(run_root: Path, model_id: str):
    run_root.mkdir(parents=True, exist_ok=True)
    with open(run_root / "env_manifest.txt", "w", encoding="utf-8") as f:
        for k, v in env_manifest(model_id).items():
            f.write(f"{k}={v}\n")
    files = []
    for r, _, fs in os.walk(run_root):
        for fn in fs:
            p = Path(r) / fn
            if p.is_file():
                h = hashlib.sha256()
                with open(p, "rb") as fh:
                    for chunk in iter(lambda: fh.read(1 << 20), b""):
                        h.update(chunk)
                rel = str(p.relative_to(run_root)).replace("\\","/")
                files.append({"path": rel, "sha256": h.hexdigest()})
    (run_root / "file_hashes.json").write_text(json.dumps({"files": files}, indent=2), encoding="utf-8")

def write_methods_note(run_root: Path, model_id: str, features: List[str], cadence: str, label: Optional[str] = None):
    note = io.StringIO()
    label_str = f"{label}" if label else model_id
    note.write(f"{label_str}\n")
    note.write("----\n")
    note.write("This price-only LSTM ablation follows the project invariants.\n")
    note.write(f"Inputs: {', '.join(features)}; Target: next-day Close (Close.shift(-1)).\n")
    note.write("Scaling: Train-only z-scaling of X; y is scaled internally and inverse-transformed for metrics.\n")
    note.write(f"Cadence: {cadence} on Test (frozen weights). Residual means can be mildly non-zero under drift.\n")
    note.write("Splits: Train 2021-02-03 → 2022-12-30; Val 2023-01-03 → 2023-05-31; Test 2023-06-01 → 2023-12-28.\n")
    note.write("Artefacts: predictions, metrics, run_config, training curves, env manifest, file hashes.\n")
    (run_root / "METHODS_NOTE.txt").write_text(note.getvalue(), encoding="utf-8")

def zip_out(root_dir: Path, zip_name: str):
    z = Path(zip_name)
    if z.exists():
        z.unlink()
    shutil.make_archive(base_name=str(z).replace(".zip",""), format="zip", root_dir=str(root_dir))
    print(f"[zip] {z.resolve()}")

## Data Loading

What: Load per-ticker panels, normalise column names, enforce date sorting, and assert the Test window size.
Why: Guarantees schema compatibility and correct temporal coverage (n = 146).
Method choices: Fixed splits as above; Target = Close.shift(-1) is used downstream; America/New_York 16:00 discipline is recorded in manifests; scaling is Train-only; deep ablations are frozen on Test.

In [ ]:
def load_panel(ticker: str) -> pd.DataFrame:
    p = INPUT_DIR / f"{ticker}_input.csv"
    if not p.exists():
        raise FileNotFoundError(f"Missing input file: {p}")
    df = pd.read_csv(p)
    if "Date" in df.columns and "date" not in df.columns:
        df = df.rename(columns={"Date": "date"})
        df.to_csv(p, index=False)
    for c in REQUIRED_COLS:
        if c not in df.columns:
            raise ValueError(f"{p.name} missing required column: {c}")
    df["date"] = pd.to_datetime(df["date"])
    df = df[df["ticker"] == ticker].sort_values("date").reset_index(drop=True)
    return df

def mask_window(df: pd.DataFrame, start: str, end: str) -> pd.Series:
    return (df["date"] >= pd.to_datetime(start)) & (df["date"] <= pd.to_datetime(end))

def assert_test_days(df: pd.DataFrame):
    n = int(mask_window(df, TEST_START, TEST_END).sum())
    assert n == TEST_DAYS_EXPECTED, f"Test days != {TEST_DAYS_EXPECTED}. Got {n}."

## Preprocessing

What: Fit Train-only scalers and build causal sequences for Train/Val windows; define dataset wrapper.
Why: Ensures no leakage and consistent input shapes (L = 90, horizon 1).
Method choices: Train-only z-scaling; zero-preservation applies only to sentiment columns (not used here); fixed splits; Target = Close.shift(-1); early stopping on Val.

In [ ]:
class SeqDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32).reshape(-1, 1)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

def fit_x_scaler_train_only(df: pd.DataFrame, features: List[str]) -> StandardScaler:
    m = mask_window(df, TRAIN_START, TRAIN_END)
    sc = StandardScaler()
    sc.fit(df.loc[m, features].values)
    return sc

def fit_y_scaler_train_only(df: pd.DataFrame, target: str) -> StandardScaler:
    m = mask_window(df, TRAIN_START, TRAIN_END)
    y = df.loc[m, target].values.reshape(-1, 1)
    sc = StandardScaler()
    sc.fit(y)
    return sc

def build_sequences(df: pd.DataFrame, x_scaler: StandardScaler, y_scaler: Optional[StandardScaler],
                    features: List[str], target: str,
                    start: str, end: str, L: int, horizon: int,
                    scale_y: bool) -> Tuple[np.ndarray, np.ndarray, List[int]]:
    Xs, ys, idxs = [], [], []
    F_scaled = x_scaler.transform(df[features].values)
    y_level = df[target].values
    dates = df["date"].values
    for t in range(L - 1, len(df) - horizon):
        t_label = t + horizon
        d_label = dates[t_label]
        if (d_label >= np.datetime64(start)) and (d_label <= np.datetime64(end)):
            Xs.append(F_scaled[t - L + 1:t + 1, :])
            y_val = y_level[t_label]
            if scale_y:
                y_val = float(y_scaler.transform(np.array([[y_val]])).ravel()[0])
            ys.append(y_val)
            idxs.append(t)
    if not Xs:
        return np.empty((0, L, len(features))), np.empty((0,)), []
    return np.stack(Xs, axis=0), np.array(ys), idxs

## Model Definition

What: Define the stacked LSTM regressor with last-timestep linear head; utility to count parameters.
Why: Establishes the price-only ablation model used as a frozen-weights baseline.
Method choices: No bidirectional layers; conservative hidden size; deterministic initialisation.

In [ ]:
class LSTMPrice(nn.Module):
    def __init__(self, input_size: int, hidden_size: int = 64, num_layers: int = 2,
                 dropout: float = 0.20, bidirectional: bool = False):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
            bidirectional=bidirectional
        )
        d = hidden_size * (2 if bidirectional else 1)
        self.head = nn.Linear(d, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        yhat = self.head(last)
        return yhat

def count_params(model: nn.Module) -> int:
    return int(sum(p.numel() for p in model.parameters() if p.requires_grad))

## Training

What: Epoch loop with Adam, MSE loss, gradient clipping, and early stopping on Validation; returns best state.
Why: Prevents overfitting and locks the model at the best validation epoch before the frozen Test pass.
Method choices: Early stopping on Val only; seeds fixed; loaders deterministic.

In [ ]:
@dataclass
class TrainHistory:
    train_loss: List[float]
    val_loss: List[float]
    best_epoch: int
    best_val: float

def run_epoch(model, loader, loss_fn, device, optim=None, grad_clip=None):
    losses = []
    if optim is None:
        model.eval()
        with torch.no_grad():
            for xb, yb in loader:
                xb, yb = xb.to(device), yb.to(device)
                yhat = model(xb)
                losses.append(loss_fn(yhat, yb).item())
        return float(np.mean(losses)) if losses else float("nan")
    else:
        model.train()
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            optim.zero_grad()
            yhat = model(xb)
            loss = loss_fn(yhat, yb)
            loss.backward()
            if grad_clip is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip)
            optim.step()
            losses.append(loss.item())
        return float(np.mean(losses)) if losses else float("nan")

def train_with_early_stopping(model: nn.Module,
                              train_loader: DataLoader,
                              val_loader: DataLoader,
                              max_epochs: int = 300,
                              lr: float = 1e-3,
                              patience: int = 12,
                              grad_clip: float = 0.5,
                              device: str = "cpu") -> TrainHistory:
    model = model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    best_state, best_val = None, float("inf")
    best_epoch, no_improve = -1, 0
    train_hist, val_hist = [], []
    for epoch in range(1, max_epochs + 1):
        tr = run_epoch(model, train_loader, loss_fn, device, optim=opt, grad_clip=grad_clip)
        va = run_epoch(model, val_loader,   loss_fn, device, optim=None)
        train_hist.append(tr); val_hist.append(va)
        if va < best_val - 1e-8:
            best_val, best_epoch, no_improve = va, epoch, 0
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        else:
            no_improve += 1
        if no_improve >= patience:
            break
    if best_state is not None:
        model.load_state_dict(best_state)
    return TrainHistory(train_hist, val_hist, best_epoch, best_val)

##Evaluation

What: Compute point-forecast metrics (RMSE/MAE/U2), ε-gated DA, pure-sign trading diagnostics (Sharpe/MaxDD/Turnover), and walk-forward inference under frozen weights.
Why: Quantifies accuracy vs naïve, directional skill with ε-gating, and realised trading diagnostics using the stated rule.
Method choices: DA uses ε = 0.0010; trading uses pure-sign positions with costs on changes; returns are percentage; frozen weights on Test (deep); expanding-origin cadence applies to ARIMA/ARIMAX, not here.

In [ ]:
def rmse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def mae(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.mean(np.abs(y_true - y_pred)))

def theils_u2(y_true: np.ndarray, y_pred: np.ndarray, y_last: np.ndarray) -> float:
    num = np.sqrt(np.mean((y_true - y_pred) ** 2))
    den = np.sqrt(np.mean((y_true - y_last) ** 2))
    return float(num / den) if den > 0 else float("inf")

def directional_accuracy(y_true: np.ndarray, y_pred: np.ndarray, y_last: np.ndarray,
                         epsilon_ret: float = 0.0010) -> Dict[str, float]:
    r_true = (y_true - y_last) / np.where(y_last==0.0, 1.0, y_last)
    r_pred = (y_pred - y_last) / np.where(y_last==0.0, 1.0, y_last)
    mask = np.abs(r_true) > epsilon_ret
    n = int(mask.sum())
    if n == 0:
        return {"DA": float("nan"), "coverage": 0.0, "n": 0}
    agree = np.sign(r_true[mask]) == np.sign(r_pred[mask])
    return {"DA": float(np.mean(agree)), "coverage": float(n / len(r_true)), "n": n}

def sharpe_and_maxdd(returns: np.ndarray) -> Tuple[float, float]:
    if returns.size == 0:
        return float("nan"), float("nan")
    mu = np.mean(returns)
    sd = np.std(returns, ddof=1)
    sharpe = float(mu / sd) if sd > 0 else float("nan")
    equity = (1.0 + returns).cumprod()
    peak = np.maximum.accumulate(equity)
    maxdd = float(np.max(1.0 - equity / peak))
    return sharpe, maxdd

def direction_only_returns(y_true: np.ndarray, y_pred: np.ndarray, y_last: np.ndarray,
                           epsilon_ret: float, bps_cost: float) -> Tuple[np.ndarray, np.ndarray]:
    r_true = (y_true - y_last) / np.where(y_last==0.0, 1.0, y_last)
    r_pred = (y_pred - y_last) / np.where(y_last==0.0, 1.0, y_last)
    pos = np.sign(r_pred).astype(float)
    pos_prev = np.roll(pos, 1); pos_prev[0] = 0.0
    changed = (pos != pos_prev).astype(float)
    tc = (bps_cost / 10000.0) * changed
    strat = pos * r_true - tc
    return strat, (changed > 0)

def walkforward_predict(df: pd.DataFrame, x_scaler: StandardScaler, y_scaler: StandardScaler,
                        model: nn.Module, features: List[str], L: int) -> pd.DataFrame:
    model.eval()
    device = next(model.parameters()).device
    dates = df["date"].values
    close = df["Close"].values
    F_scaled = x_scaler.transform(df[features].values)
    rows = []
    for t in range(L - 1, len(df) - HORIZON):
        t_label = t + HORIZON
        d_label = dates[t_label]
        if (d_label < np.datetime64(TEST_START)) or (d_label > np.datetime64(TEST_END)):
            continue
        x_win = F_scaled[t - L + 1:t + 1, :]
        xb = torch.from_numpy(x_win.astype(np.float32)).unsqueeze(0).to(device)
        with torch.no_grad():
            yhat_scaled = model(xb).cpu().numpy().ravel()[0]
        yhat_level = float(y_scaler.inverse_transform(np.array([[yhat_scaled]])).ravel()[0])
        rows.append((pd.to_datetime(d_label), close[t_label], yhat_level, close[t]))
    pred_df = pd.DataFrame(rows, columns=["date","y_true","y_hat","y_last"]).sort_values("date").reset_index(drop=True)
    pred_df["residual"] = pred_df["y_true"] - pred_df["y_hat"]
    pred_df["in_sample_flag"] = 0
    return pred_df

## Outputs and Artefacts

What: Emit predictions/metrics/run_config/training curves per ticker; write run-root manifests; package bundles.
Why: Produces reproducible artefacts for dissertation and audit without behaviour changes.
Method choices: Metrics on inverse-transformed level values; DA uses ε = 0.0010; trading diagnostics use pure-sign rule with costs on changes; deep ablation cadence is frozen weights (no monthly refit; expanding-origin applies to ARIMA/ARIMAX only).

In [ ]:
def emit_run_config(model_id: str, ticker: str, out_dir: Path, features: List[str],
                    param_count: int, label: Optional[str] = None) -> Path:
    cfg = {
        "model_id": model_id,
        "model_label": label if label else model_id,
        "ticker": ticker,
        "features_used": features,
        "parameter_count": int(param_count),
        "seeds": {"global": SEED},
        "cadence": "frozen_weights",
        "window_length": LOOKBACK_L,
        "horizon": HORIZON,
        "batch_size": BATCH_TRAIN,
        "epochs_max": MAX_EPOCHS,
        "timezone": "America/New_York",
        "market_close": "16:00",
        "policy": {
            "x_scaler": "train_only_z",
            "y_scaled": True,
            "inverse_transform_metrics": True
        },
        "target_definition": "Close.shift(-1) created after all features",
        "target_col": "Close",
        "splits": {
            "train": [TRAIN_START, TRAIN_END],
            "val":   [VAL_START, VAL_END],
            "test":  [TEST_START, TEST_END]
        }
    }
    p = out_dir / f"run_config_{model_id}_{ticker}.json"
    p.write_text(json.dumps(cfg, indent=2), encoding="utf-8")
    return p

def training_curves_png(history: TrainHistory, out_path: Path, ticker: str, model_id: str):
    fig = plt.figure(figsize=(7,4))
    plt.plot(history.train_loss, label="train_loss")
    plt.plot(history.val_loss,   label="val_loss")
    plt.xlabel("epoch"); plt.ylabel("loss")
    plt.title(f"{model_id} {ticker} (best_epoch={history.best_epoch})")
    plt.legend(); fig.tight_layout(); fig.savefig(out_path, dpi=160); plt.close(fig)

def process_ticker(model_id: str, ticker: str, model_dir: Path, features: List[str],
                   model_label: Optional[str] = None) -> Dict[str, object]:
    set_all_seeds(SEED)
    df = load_panel(ticker)
    assert_test_days(df)
    x_scaler = fit_x_scaler_train_only(df, features)
    y_scaler = fit_y_scaler_train_only(df, "Close")
    X_tr, y_tr, _ = build_sequences(df, x_scaler, y_scaler, features, "Close",
                                    TRAIN_START, TRAIN_END, LOOKBACK_L, HORIZON, scale_y=True)
    X_va, y_va, _ = build_sequences(df, x_scaler, y_scaler, features, "Close",
                                    VAL_START, VAL_END, LOOKBACK_L, HORIZON, scale_y=True)
    train_loader = DataLoader(SeqDataset(X_tr, y_tr), batch_size=BATCH_TRAIN, shuffle=False, drop_last=False)
    val_loader   = DataLoader(SeqDataset(X_va, y_va), batch_size=BATCH_VAL,   shuffle=False, drop_last=False)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = LSTMPrice(input_size=len(features), hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS,
                      dropout=DROPOUT, bidirectional=BIDIR).to(device)
    n_params = count_params(model)
    hist = train_with_early_stopping(model, train_loader, val_loader,
                                     max_epochs=MAX_EPOCHS, lr=LR, patience=PATIENCE,
                                     grad_clip=GRAD_CLIP, device=device)
    preds_df = walkforward_predict(df, x_scaler, y_scaler, model, features, LOOKBACK_L)
    assert len(preds_df) == TEST_DAYS_EXPECTED, f"{ticker}: predictions rows != {TEST_DAYS_EXPECTED}"
    y_true = preds_df["y_true"].to_numpy()
    y_hat  = preds_df["y_hat"].to_numpy()
    y_last = preds_df["y_last"].to_numpy()
    residual = y_true - y_hat
    m_rmse = rmse(y_true, y_hat)
    m_mae  = mae(y_true, y_hat)
    m_u2   = theils_u2(y_true, y_hat, y_last)
    da     = directional_accuracy(y_true, y_hat, y_last, epsilon_ret=EPSILON_RET)
    strat0, changed0  = direction_only_returns(y_true, y_hat, y_last, EPSILON_RET, 0.0)
    strat10, changed10 = direction_only_returns(y_true, y_hat, y_last, EPSILON_RET, 10.0)
    s0, mdd0   = sharpe_and_maxdd(strat0)
    s10, mdd10 = sharpe_and_maxdd(strat10)
    turnover = int(np.sum(changed0))
    tdir = model_dir / ticker
    tdir.mkdir(parents=True, exist_ok=True)
    pred_cols = ["date","y_true","y_hat","residual","in_sample_flag"]
    preds_out = preds_df[pred_cols].copy()
    preds_out["in_sample_flag"] = preds_out["in_sample_flag"].astype(int)
    pred_path = tdir / f"predictions_{model_id}_{ticker}.csv"
    preds_out.to_csv(pred_path, index=False)
    metrics = {
        "RMSE": float(m_rmse),
        "MAE": float(m_mae),
        "U2": float(m_u2),
        "DA_epsilon": float(da["DA"]),
        "Coverage": float(da["coverage"]),
        "n": int(TEST_DAYS_EXPECTED),
        "Sharpe_0bps": float(s0),
        "Sharpe_10bps": float(s10),
        "MaxDD_0bps": float(mdd0),
        "MaxDD_10bps": float(mdd10),
        "Turnover": turnover,
        "ResidualMean": float(np.mean(residual)),
        "ResidualMeanAbs": float(np.mean(np.abs(residual))),
        "ResidualStd": float(np.std(residual, ddof=1)),
        "trading_rule": "pure_sign",
        "tie_policy": "zero",
        "return_type": "pct",
        "epsilon_DA": EPSILON_RET,
        "cost_bps": [0, 10]
    }
    metrics_path = tdir / f"metrics_{model_id}_{ticker}.json"
    metrics_path.write_text(json.dumps(metrics, indent=2), encoding="utf-8")
    curves_path = tdir / f"training_curves_{model_id}_{ticker}.png"
    training_curves_png(hist, curves_path, ticker, model_id)
    run_cfg_path = emit_run_config(model_id, ticker, tdir, features, n_params, label=model_label)
    fig = plt.figure(figsize=(9,5))
    plt.plot(preds_df["date"], preds_df["y_true"], label="actual")
    plt.plot(preds_df["date"], preds_df["y_hat"],  label="predicted")
    plt.legend(); plt.grid(True, linestyle=":")
    fig.tight_layout(); fig.savefig(tdir / f"actual_vs_pred_{model_id}_{ticker}.png", dpi=140); plt.close(fig)
    return {
        "predictions_csv": str(pred_path),
        "metrics_json": str(metrics_path),
        "run_config_json": str(run_cfg_path),
        "training_curves_png": str(curves_path)
    }

def run_suite(model_id: str, features: List[str], run_root_name: str, bundle_name: str,
              model_label: Optional[str] = None):
    set_all_seeds(SEED)
    RUN_ROOT = Path(run_root_name)
    MODEL_DIR = RUN_ROOT / model_id
    MODEL_DIR.mkdir(parents=True, exist_ok=True)
    for t in TICKERS:
        print(f"=== {t} ({features}) -> {model_id} ===")
        arte = process_ticker(model_id, t, MODEL_DIR, features, model_label=model_label)
        for key in ["predictions_csv","metrics_json","run_config_json","training_curves_png"]:
            p = Path(arte[key]); assert p.exists() and p.stat().st_size > 0, f"Missing: {p}"
    write_methods_note(RUN_ROOT, model_id, features, cadence="frozen_weights", label=model_label)
    write_run_root_manifests(RUN_ROOT, model_id)
    zip_out(RUN_ROOT, bundle_name)
    downloaded = False
    try:
        from google.colab import files
        files.download(bundle_name)
        downloaded = True
    except Exception:
        pass
    if not downloaded:
        try:
            from IPython.display import FileLink, display
            display(FileLink(bundle_name))
        except Exception:
            print(f"[saved] {Path(bundle_name).resolve()}")

if __name__ == "__main__":
    run_suite(
        model_id="LSTM_PO",
        features=["Open","High","Low","Close","Volume"],
        run_root_name="LSTM_PO_FINAL_OHLCV",
        bundle_name="LSTM_PO_FINAL_OHLCV_bundle.zip",
        model_label="LSTM_PO (price-only OHLCV)"
    )
    run_suite(
        model_id="LSTM_PO_CloseOnly",
        features=["Close"],
        run_root_name="LSTM_PO_CloseOnly_ABL",
        bundle_name="LSTM_PO_CloseOnly_ablation_bundle.zip",
        model_label="LSTM_PO_CloseOnly (ablation)"
    )

=== AAPL (['Open', 'High', 'Low', 'Close', 'Volume']) -> LSTM_PO ===
=== AMZN (['Open', 'High', 'Low', 'Close', 'Volume']) -> LSTM_PO ===
=== MSFT (['Open', 'High', 'Low', 'Close', 'Volume']) -> LSTM_PO ===
=== TSLA (['Open', 'High', 'Low', 'Close', 'Volume']) -> LSTM_PO ===
=== AMD (['Open', 'High', 'Low', 'Close', 'Volume']) -> LSTM_PO ===
[zip] /content/LSTM_PO_FINAL_OHLCV_bundle.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

=== AAPL (['Close']) -> LSTM_PO_CloseOnly ===
=== AMZN (['Close']) -> LSTM_PO_CloseOnly ===
=== MSFT (['Close']) -> LSTM_PO_CloseOnly ===
=== TSLA (['Close']) -> LSTM_PO_CloseOnly ===
=== AMD (['Close']) -> LSTM_PO_CloseOnly ===
[zip] /content/LSTM_PO_CloseOnly_ablation_bundle.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>